In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
from utils import check_paths
import pandas as pd
from scipy.io import loadmat
import joblib
import matplotlib.gridspec as gridspec

from pactools import Comodulogram, REFERENCES, raw_to_mask

from mne.channels.layout import find_layout
from functools import partial
from mne.defaults import _handle_default

from mne.viz.topo import _erfimage_imshow_unified, _plot_topo

from mne.viz.utils import (
    _setup_vmin_vmax,
    add_background_image
)
from collections import namedtuple

from mne.stats import permutation_cluster_1samp_test

import scipy
from scipy.stats import zscore

%matplotlib qt

In [9]:
src_dir = 'D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y'
src_fname = os.path.join(src_dir, 'fsaverage_bem\\bem\\fsaverage-ico4-src.fif')
src = mne.read_source_spaces(src_fname)
# src.plot(subjects_dir='D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y')
source_adjacency = mne.spatial_src_adjacency(src)
adj = source_adjacency.tocsr()  # Ensure adjacency is CSR format for fast indexing
print('adjacency shape:', source_adjacency.shape)

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
-- number of adjacent vertices : 5124
adjacency shape: (5124, 5124)


In [ ]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
group = 'Y'
subs = os.listdir(os.path.join(eeg_data_dir, group))
sub_name = 's1_pac_sub01'
task = '_BL'
task_stage = '_plan'


TypeError: __init__() missing 2 required positional arguments: 'tmin' and 'tstep'

In [16]:
source_dir = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis', 'source')
morphed_dir = os.path.join(source_dir, 'morphed_stcs')
i = '001'
stc = mne.read_source_estimate(os.path.join(morphed_dir, task, task_stage, f'{sub_name}{task}{task_stage}-stc-lcmv_epoch_{i}_morphed'))

In [ ]:
# Option 1: Standard labels from aparc
brain = stc.plot(subject='fsaverage_bem', subjects_dir=src_dir, hemi='split',
                 surface='inflated', time_viewer=True, views=['lat', 'med'],
                 clim='auto', colormap='magma')

# Load labels
labels = mne.read_labels_from_annot('fsaverage_bem', parc='aparc', subjects_dir=src_dir)

roi_names = [
    # Primary motor cortex
    'precentral-lh', 'precentral-rh',
    # Premotor cortex
    'caudalmiddlefrontal-lh', 'caudalmiddlefrontal-rh',
    # Supplementary motor area
    'superiorfrontal-lh', 'superiorfrontal-rh',
    # Primary somatosensory cortex
    'postcentral-lh', 'postcentral-rh',
    # Prefrontal cortex
    'rostralmiddlefrontal-lh', 'rostralmiddlefrontal-rh',
]

roi_labels = [lab for lab in labels if lab.name in roi_names]

# Overlay labels on the brain
for lab in roi_labels:
    brain.add_label(lab, borders=True)

Using control points [0.24671839 0.30328944 0.88228696]
Reading labels from parcellation...
   read 35 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\lh.aparc.annot
   read 34 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\rh.aparc.annot


In [ ]:
# OPTION 2: More detailed labels from aparc.a2009s
brain = stc.plot(subject='fsaverage_bem', subjects_dir=src_dir, hemi='split',
                 surface='inflated', time_viewer=True, views=['lat', 'med'],
                 clim='auto', colormap='magma')

# Load labels
labels = mne.read_labels_from_annot('fsaverage_bem', parc='aparc.a2009s', subjects_dir=src_dir)

roi_labels = [
    # Primary Motor Cortex
    'G_precentral-lh', 'G_precentral-rh',

    # Premotor Cortex
    'S_precentral-sup-part-lh', 'S_precentral-sup-part-rh',
    'S_precentral-inf-part-lh', 'S_precentral-inf-part-rh',
    'G_front_middle-lh', 'G_front_middle-rh',

    # Supplementary Motor Area
    'G_and_S_paracentral-lh', 'G_and_S_paracentral-rh',
    'G_front_sup-lh', 'G_front_sup-rh',

    # Primary Somatosensory Cortex
    'G_postcentral-lh', 'G_postcentral-rh',
    'S_postcentral-lh', 'S_postcentral-rh',

    # Prefrontal Cortex
    'S_front_sup-lh', 'S_front_sup-rh',
    'S_front_inf-lh', 'S_front_inf-rh',
    'S_front_middle-lh', 'S_front_middle-rh'
]

roi_labels = [lab for lab in labels if lab.name in roi_labels]

# Overlay labels on the brain
for lab in roi_labels:
    brain.add_label(lab, borders=True)

Using control points [0.24671839 0.30328944 0.88228696]
Reading labels from parcellation...
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\lh.aparc.a2009s.annot
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\rh.aparc.a2009s.annot


In [46]:
method = 'mean_flip'  # average signal considering dipole orientation
roi_tc = extract_label_time_course(stc, roi_labels, src, mode=method)

Extracting time courses for 22 labels (mode: mean_flip)


In [48]:
# Suppose we want to visualize all ROI time courses together
# Concatenate both hemispheres
n_labels = len(roi_labels)
n_times = roi_tc.shape[1]

# Create a dummy vertex mapping (one per ROI for each hemisphere)
vertices = [np.arange(n_labels // 2), np.arange(n_labels // 2)]

# Create the SourceEstimate
roi_stc = mne.SourceEstimate(roi_tc, vertices, stc.tmin, stc.tstep, subject=stc.subject)

In [50]:
brain = roi_stc.plot(
    subject='fsaverage_bem',
    subjects_dir=src_dir,
    hemi='split',
    views='lat',
    time_viewer=True,
    smoothing_steps=5,
    clim='auto'
)

Using control points [0.15532778 0.1774492  0.35075434]


Using control points [0.21868608 0.25945783 0.3260517 ]
